1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np
import random
import datetime as dt

file_name = "data_000637.txt" 
N=random.randint(10000,1300000)
data = pd.read_csv(file_name ,nrows=N)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
893506,1,1,11,3869208296,1032,14
893507,1,1,13,3869208296,1034,6
893508,1,0,124,3869208296,1038,15
893509,1,1,139,3869208296,1042,0


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
data.groupby(by="ORBIT_CNT")["BX_COUNTER"].transform(np.max).unique().mean()

2559.060524546066

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [6]:
data["ABS_TIME"]=(data["BX_COUNTER"]*25+data["TDC_MEAS"]*(5/6)).cumsum()
data
#Con le due righe di seguito avrei individuato una series contente il cumulativo del tempo
#time_column=pd.Series(data["ABS_TIME"])
#time_column
data["Total"]=pd.to_datetime(data["ABS_TIME"],unit="ns")
data

0         5.937167e+04
1         1.187442e+05
2         1.825925e+05
3         2.465583e+05
4         3.155792e+05
              ...     
893506    3.994230e+10
893507    3.994232e+10
893508    3.994235e+10
893509    3.994237e+10
893510    3.994240e+10
Name: ABS_TIME, Length: 893511, dtype: float64

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [ ]:
s=pd.Series([data.iloc[0,7],data.iloc[-1,7]])
s.diff()

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [ ]:
data.groupby(by=["FPGA","TDC_CHANNEL"])["FPGA"].count().reset_index(name='count').sort_values(["count"],ascending=False)

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [ ]:
len(data.groupby("ORBIT_CNT")["ORBIT_CNT"].count())

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [ ]:
len(data.loc[data['TDC_CHANNEL']==139]["ORBIT_CNT"].unique())

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [ ]:
a=data.loc[data['FPGA']==0].groupby("TDC_CHANNEL")["TDC_CHANNEL"].count()
b=data.loc[data['FPGA']==1].groupby("TDC_CHANNEL")["TDC_CHANNEL"].count()
print(a)
print(b)

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.